<a href="https://colab.research.google.com/github/kameda-yoshinari/IMISToolExeA2022/blob/main/600/pytorch_advanced-revised/7_nlp_sentiment_transformer/GC7_7_transformer_training_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 7.7 Transformerの学習・推論、判定根拠の可視化を実装

- 本ファイルでは、ここまでで作成したTransformerモデルとIMDbのDataLoaderを使用してクラス分類を学習させます。テストデータで推論をし、さらに判断根拠となるAttentionを可視化します

# 7.7 学習目標

1.	Transformerの学習を実装できるようになる
2.	Transformerの判定時のAttention可視化を実装できるようになる


---

# Google Colab

In [ ]:
!echo "Change to the JST notation."
!rm /etc/localtime
!ln -s /usr/share/zoneinfo/Japan /etc/localtime

Change to the JST notation.


In [ ]:
!echo "Start mounting your Google Drive."
from google.colab import drive 
drive.mount('/content/drive')
%cd /content/drive/My\ Drive/
!echo "Move to the working directory."
%cd IMIS_Tool-A/Work600/
!ls -l

Start mounting your Google Drive.
Mounted at /content/drive
/content/drive/My Drive
Move to the working directory.
/content/drive/My Drive/IMIS_Tool-A/Work600
total 4
drwx------ 2 root root 4096 Aug  7 22:01 pytorch_advanced


---
# 共通準備

"pytorch_advanced" folder should be ready before you come here.

In [ ]:
# Skip this if you have already issued git in advance. 
# If you come here by way of 600-PyTorchADL.ipynb, 
# you should skip the git command (as you have already issued in 600).  
# If you run git when pytorch_advanced already exists, git tells the error and clone won't be made.

#!git clone https://github.com/YutaroOgawa/pytorch_advanced.git

import os
if os.path.exists("/content/drive/My Drive/IMIS_Tool-A/Work600/pytorch_advanced"):
    print("OK. Alreadly git cloned. You can go.")
else:
    print("You'd better go back to the first 600-PyTorchADL.ipynb")

OK. Alreadly git cloned. You can go.


In [ ]:
!ls

pytorch_advanced


In [ ]:
%cd "pytorch_advanced"

/content/drive/My Drive/IMIS_Tool-A/Work600/pytorch_advanced


In [ ]:
!ls

1_image_classification	 7_nlp_sentiment_transformer
2_objectdetection	 8_nlp_sentiment_bert
3_semantic_segmentation  9_video_classification_eco
4_pose_estimation	 etc
5_gan_generation	 LICENSE
6_gan_anomaly_detection  README.md


In [ ]:
%cd "7_nlp_sentiment_transformer"

/content/drive/My Drive/IMIS_Tool-A/Work600/pytorch_advanced/7_nlp_sentiment_transformer


In [ ]:
!ls

7-1_Tokenizer.ipynb		   7-7_transformer_training_inference.ipynb
7-2_torchtext.ipynb		   data
7-4_vectorize.ipynb		   make_folders_and_data_downloads.ipynb
7-5_IMDb_Dataset_DataLoader.ipynb  utils
7-6_Transformer.ipynb


---
# Extraction of aclImdb (GC7-0)



In [ ]:
# local drive on GC is very, very fast.
!mkdir -p /root/data/
%cd /root/data/
!tar xfz '/content/drive/My Drive/IMIS_Tool-A/Work600/pytorch_advanced/7_nlp_sentiment_transformer/data/aclImdb_v1.tar.gz'

# make a symbolic link at the working directory on google drive.
%cd '/content/drive/My Drive/IMIS_Tool-A/Work600/pytorch_advanced/7_nlp_sentiment_transformer/'
!rm -f data/aclImdb
!ln -s /root/data/aclImdb data/aclImdb

!ls -ld data/aclImdb/

/root/data
/content/drive/My Drive/IMIS_Tool-A/Work600/pytorch_advanced/7_nlp_sentiment_transformer
drwxr-xr-x 4 7297 1000 4096 Jun 26  2011 data/aclImdb/


# utils/dataloader.py

utils/dataloader.py should be updated on 7-6_Transformer.ipynb .

In [ ]:
!head -5 utils/dataloader.py

# 第7章 自然言語処理による感情分析（Transformer）


import glob
import os


In [ ]:
# We use torchtext version 0.8 (and torch 1.7.1) that was popular when the book was published. See issue 148.
# https://github.com/YutaroOgawa/pytorch_advanced/issues/148
import warnings
warnings.simplefilter('ignore')

!pip uninstall -y torchtext torchvision torchaudio
!pip install torch==1.7.1+cu110 -f https://download.pytorch.org/whl/torch_stable.html
!pip install torchtext==0.8

Found existing installation: torchtext 0.13.0
Uninstalling torchtext-0.13.0:
  Successfully uninstalled torchtext-0.13.0
Found existing installation: torchvision 0.13.0+cu113
Uninstalling torchvision-0.13.0+cu113:
  Successfully uninstalled torchvision-0.13.0+cu113
Found existing installation: torchaudio 0.12.0+cu113
Uninstalling torchaudio-0.12.0+cu113:
  Successfully uninstalled torchaudio-0.12.0+cu113
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |███████████████████████         | 834.1 MB 1.2 MB/s eta 0:04:29tcmalloc: large alloc 1147494400 bytes == 0x3954c000 @  0x7ff315032615 0x592b76 0x4df71e 0x59afff 0x515655 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x593fce 0x548ae9 0x5127f1 0x598e3b 0x511f68 0x598e3b 0x511f68 0x598e3b 0x511f68 0x4bc98a 0x532e76 0x594b72 0x515600 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x593fce 0x5118f8 0x593dd7
     |████████

---
# 7.7節


In [ ]:
# パッケージのimport
import numpy as np
import random

import torch
import torch.nn as nn
import torch.optim as optim

import torchtext


In [ ]:
# 乱数のシードを設定
torch.manual_seed(1234)
np.random.seed(1234)
random.seed(1234)

# DatasetとDataLoaderを作成

In [ ]:
# it takes about one minute.

from utils.dataloader import get_IMDb_DataLoaders_and_TEXT

# 読み込み
train_dl, val_dl, test_dl, TEXT = get_IMDb_DataLoaders_and_TEXT(
    max_length=256, batch_size=64)

# 辞書オブジェクトにまとめる
dataloaders_dict = {"train": train_dl, "val": val_dl}


# ネットワークモデルの作成

In [ ]:
from utils.transformer import TransformerClassification

# モデル構築
net = TransformerClassification(
    text_embedding_vectors=TEXT.vocab.vectors, d_model=300, max_seq_len=256, output_dim=2)

# ネットワークの初期化を定義


def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Linear') != -1:
        # Liner層の初期化
        nn.init.kaiming_normal_(m.weight)
        if m.bias is not None:
            nn.init.constant_(m.bias, 0.0)


# 訓練モードに設定
net.train()

# TransformerBlockモジュールを初期化実行
net.net3_1.apply(weights_init)
net.net3_2.apply(weights_init)


print('ネットワーク設定完了')


ネットワーク設定完了


# 損失関数と最適化手法を定義

In [ ]:
# 損失関数の設定
criterion = nn.CrossEntropyLoss()
# nn.LogSoftmax()を計算してからnn.NLLLoss(negative log likelihood loss)を計算

# 最適化手法の設定
learning_rate = 2e-5
optimizer = optim.Adam(net.parameters(), lr=learning_rate)


# 学習・検証を実施

In [ ]:
# モデルを学習させる関数を作成


def train_model(net, dataloaders_dict, criterion, optimizer, num_epochs):

    # GPUが使えるかを確認
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("使用デバイス：", device)
    print('-----start-------')
    # ネットワークをGPUへ
    net.to(device)

    # ネットワークがある程度固定であれば、高速化させる
    torch.backends.cudnn.benchmark = True

    # epochのループ
    for epoch in range(num_epochs):
        # epochごとの訓練と検証のループ
        for phase in ['train', 'val']:
            if phase == 'train':
                net.train()  # モデルを訓練モードに
            else:
                net.eval()   # モデルを検証モードに

            epoch_loss = 0.0  # epochの損失和
            epoch_corrects = 0  # epochの正解数

            # データローダーからミニバッチを取り出すループ
            for batch in (dataloaders_dict[phase]):
                # batchはTextとLableの辞書オブジェクト

                # GPUが使えるならGPUにデータを送る
                inputs = batch.Text[0].to(device)  # 文章
                labels = batch.Label.to(device)  # ラベル

                # optimizerを初期化
                optimizer.zero_grad()

                # 順伝搬（forward）計算
                with torch.set_grad_enabled(phase == 'train'):

                    # mask作成
                    input_pad = 1  # 単語のIDにおいて、'<pad>': 1 なので
                    input_mask = (inputs != input_pad)

                    # Transformerに入力
                    outputs, _, _ = net(inputs, input_mask)
                    loss = criterion(outputs, labels)  # 損失を計算

                    _, preds = torch.max(outputs, 1)  # ラベルを予測

                    # 訓練時はバックプロパゲーション
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                    # 結果の計算
                    epoch_loss += loss.item() * inputs.size(0)  # lossの合計を更新
                    # 正解数の合計を更新
                    epoch_corrects += torch.sum(preds == labels.data)

            # epochごとのlossと正解率
            epoch_loss = epoch_loss / len(dataloaders_dict[phase].dataset)
            epoch_acc = epoch_corrects.double(
            ) / len(dataloaders_dict[phase].dataset)

            print('Epoch {}/{} | {:^5} |  Loss: {:.4f} Acc: {:.4f}'.format(epoch+1, num_epochs,
                                                                           phase, epoch_loss, epoch_acc))

    return net


In [ ]:
# 学習・検証を実行。 (it takes 5 minutes or so on GC with GPU)
num_epochs = 10
net_trained = train_model(net, dataloaders_dict,
                          criterion, optimizer, num_epochs=num_epochs)


使用デバイス： cuda:0
-----start-------
Epoch 1/10 | train |  Loss: 0.6004 Acc: 0.6630
Epoch 1/10 |  val  |  Loss: 0.4490 Acc: 0.7958
Epoch 2/10 | train |  Loss: 0.4409 Acc: 0.7999
Epoch 2/10 |  val  |  Loss: 0.3987 Acc: 0.8244
Epoch 3/10 | train |  Loss: 0.4038 Acc: 0.8213
Epoch 3/10 |  val  |  Loss: 0.3918 Acc: 0.8342
Epoch 4/10 | train |  Loss: 0.3842 Acc: 0.8297
Epoch 4/10 |  val  |  Loss: 0.3755 Acc: 0.8434
Epoch 5/10 | train |  Loss: 0.3713 Acc: 0.8389
Epoch 5/10 |  val  |  Loss: 0.3606 Acc: 0.8486
Epoch 6/10 | train |  Loss: 0.3588 Acc: 0.8450
Epoch 6/10 |  val  |  Loss: 0.3587 Acc: 0.8510
Epoch 7/10 | train |  Loss: 0.3499 Acc: 0.8484
Epoch 7/10 |  val  |  Loss: 0.3740 Acc: 0.8462
Epoch 8/10 | train |  Loss: 0.3418 Acc: 0.8501
Epoch 8/10 |  val  |  Loss: 0.3563 Acc: 0.8520
Epoch 9/10 | train |  Loss: 0.3354 Acc: 0.8544
Epoch 9/10 |  val  |  Loss: 0.3790 Acc: 0.8426
Epoch 10/10 | train |  Loss: 0.3325 Acc: 0.8559
Epoch 10/10 |  val  |  Loss: 0.3554 Acc: 0.8544


# テストデータでの正解率を求める

In [ ]:
# device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

net_trained.eval()   # モデルを検証モードに
net_trained.to(device)

epoch_corrects = 0  # epochの正解数

for batch in (test_dl):  # testデータのDataLoader
    # batchはTextとLableの辞書オブジェクト
    
    # GPUが使えるならGPUにデータを送る
    inputs = batch.Text[0].to(device)  # 文章
    labels = batch.Label.to(device)  # ラベル

    # 順伝搬（forward）計算
    with torch.set_grad_enabled(False):

        # mask作成
        input_pad = 1  # 単語のIDにおいて、'<pad>': 1 なので
        input_mask = (inputs != input_pad)

        # Transformerに入力
        outputs, _, _ = net_trained(inputs, input_mask)
        _, preds = torch.max(outputs, 1)  # ラベルを予測

        # 結果の計算
        # 正解数の合計を更新
        epoch_corrects += torch.sum(preds == labels.data)

# 正解率
epoch_acc = epoch_corrects.double() / len(test_dl.dataset)

print('テストデータ{}個での正解率：{:.4f}'.format(len(test_dl.dataset),epoch_acc))


テストデータ25000個での正解率：0.8476


# Attentionの可視化で判定根拠を探る



In [ ]:
# HTMLを作成する関数を実装


def highlight(word, attn):
    "Attentionの値が大きいと文字の背景が濃い赤になるhtmlを出力させる関数"

    html_color = '#%02X%02X%02X' % (
        255, int(255*(1 - attn)), int(255*(1 - attn)))
    return '<span style="background-color: {}"> {}</span>'.format(html_color, word)


def mk_html(index, batch, preds, normlized_weights_1, normlized_weights_2, TEXT):
    "HTMLデータを作成する"

    # indexの結果を抽出
    sentence = batch.Text[0][index]  # 文章
    label = batch.Label[index]  # ラベル
    pred = preds[index]  # 予測

    # indexのAttentionを抽出と規格化
    attens1 = normlized_weights_1[index, 0, :]  # 0番目の<cls>のAttention
    attens1 /= attens1.max()

    attens2 = normlized_weights_2[index, 0, :]  # 0番目の<cls>のAttention
    attens2 /= attens2.max()

    # ラベルと予測結果を文字に置き換え
    if label == 0:
        label_str = "Negative"
    else:
        label_str = "Positive"

    if pred == 0:
        pred_str = "Negative"
    else:
        pred_str = "Positive"

    # 表示用のHTMLを作成する
    html = '正解ラベル：{}<br>推論ラベル：{}<br><br>'.format(label_str, pred_str)

    # 1段目のAttention
    html += '[TransformerBlockの1段目のAttentionを可視化]<br>'
    for word, attn in zip(sentence, attens1):
        html += highlight(TEXT.vocab.itos[word], attn)
    html += "<br><br>"

    # 2段目のAttention
    html += '[TransformerBlockの2段目のAttentionを可視化]<br>'
    for word, attn in zip(sentence, attens2):
        html += highlight(TEXT.vocab.itos[word], attn)

    html += "<br><br>"

    return html


In [ ]:
from IPython.display import HTML

# Transformerで処理

# ミニバッチの用意
batch = next(iter(test_dl))

# GPUが使えるならGPUにデータを送る
inputs = batch.Text[0].to(device)  # 文章
labels = batch.Label.to(device)  # ラベル

# mask作成
input_pad = 1  # 単語のIDにおいて、'<pad>': 1 なので
input_mask = (inputs != input_pad)

# Transformerに入力
outputs, normlized_weights_1, normlized_weights_2 = net_trained(
    inputs, input_mask)
_, preds = torch.max(outputs, 1)  # ラベルを予測


index = 3  # 出力させたいデータ
html_output = mk_html(index, batch, preds, normlized_weights_1,
                      normlized_weights_2, TEXT)  # HTML作成
HTML(html_output)  # HTML形式で出力


In [ ]:
index = 61  # 出力させたいデータ
html_output = mk_html(index, batch, preds, normlized_weights_1,
                      normlized_weights_2, TEXT)  # HTML作成
HTML(html_output)  # HTML形式で出力


以上

---
Revised by KAMEDA, Yoshinari at University of Tsukuba for lecture purpose.  
Original: https://github.com/YutaroOgawa/pytorch_advanced

2022/08/08.  
2021/08/03. 